## __Rigid bodies can now be added anywhere as well as arbitrary number pf patches on each rigid body.__<br>
<br>

__To Do:__<br>

- Add rotational degrees of freedom.<br>
- Figure out why moment of inertia isn't updating after being defined (stays at [0,0,0] despite being explicitly set).<br>
- Make LJ interactions rely on a dictionary/ make them user friendly to change.<br><br>

__BUG__:<br>
- For some reason the moment of inertia for a rigid body keeps resetting to [0,0,0]. I've tried two different ways to set it, and neither way workes in the main body of my code but __WILL__ work outside of the main body of my code. I have no idea what's causing this. 

In [24]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 
from inspect import currentframe

class SimBox:
    
    ###Creates Initial Snapshot
    def MakeSnapshot(self, boxsize, nparticles): 
        
        snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(L=boxsize+nparticles),
                                          bond_types=['tether'],particle_types=self.GenParticleTypes(nparticles, debug=True))
        return snapshot
    
    
    #Only used for initializing the system, do not use outside of MakeSnapshot!
    def GenParticleTypes(self, nparticles, debug=None):
        counter=0
        #List that will contain the particle types. Enable debug to print.
        GPT = [] 
        for k in range(nparticles):
            counter=k
            GPT.append('C%d'%counter)
        if(debug==True):
            print("DEBUG: GenParticleTypes: Particle Types = ",GPT)
        return GPT


In [25]:
from inspect import currentframe

class Polymer:

    def SetPolyProperties(self, nparticles, boxsize, diam=None, debug=None):
        if(diam==None):
            diam=1.0 #default diameter
        for i in range(nparticles):
            system.particles.get(i).position = [-(boxsize-nparticles)+i,0,0]
            #system.particles.get(i).moment_intertia = [0,0,0]
            system.particles.get(i).diameter = 1.0
            system.particles.get(i).type=('C%d'%(i))
            if(debug==True):
                print("DEBUG: SetPolyProperties: Particle %d is at "%i, [nparticles+i,0,0])
    
    #MakePolyTage is just a debugging tool, ignore. 
    def MakePolyTags(self, nparticles, debug=None):
        P_Tag_List = []
        for i in range(nparticles):
            system.particles.get(i).type=('C%d'%(i+1))
            P_Tag_List.append('C%d'%(i+1))
        if(debug==True):
            print("DEBUG: SetPolyProperties: P_Tag_List is ", P_Tag_List)
        
    def PatchTypes(self, tot_npatch, debug=None):
        patch_types = []
        for i in range(tot_npatch):
            system.particles.types.add('%d'%(i+1))
        if(debug==True):
            print("DEBUG: PatchTypes: ",patch_types)
        return patch_types
    
    def SetRigidProperties(self, index, moment, diam):
            system.particles.get(index).moment_intertia = moment
            system.particles.get(index).diameter = diam
            
            
#     def RigidParam(self, rigid_type, patch_positions, patch_types, pdiam=None):
#         rigid.set_param(rigid_type,
#                             positions=patch_positions, 
#                             types=patch_types,
#                             #diameters=pdiam
#                            )
    
    def MakeBonds(self, nparticles, debug=None):
        DebugBonds = []
        f=0
        for f in range(nparticles-1):
            system.bonds.add('tether',f,f+1)
            DebugBonds.append([f,f+1])
            f+=1
        if(debug==True):
            print("DEBUG: MakeBonds: Bonds = ", DebugBonds)  

In [26]:
from inspect import currentframe

class Debug:
    def PrintAllParticleTags(self, nparticles, tot_npatch):
        X = []
        for i in range(nparticles+tot_npatch):
            X.append(system.particles[i].tag)
        print("DEBUG: List of All Particle Tags: ", X)
    def PrintSysParticles(self, debug=None):
        if(debug==True):
            for i in range(len(system.particles)):
                print(system.particles[i])
    def PrintLJPairs(self, sig, types):
        for i in range(0,len(sig)):
            for j in range(i,len(sig)):
                print([types[i],types[j]])
    def DebugMessage(self, string, variable, debug=None):
        if(debug==True):
            cf = currentframe()
            print("%s"%string, variable, " at line ", cf.f_back.f_lineno)
    def DebugInfo(self):
        print("Debug Message Format Is As Follows:\n DEBUG: Function: Parameter = Value")



In [29]:
from inspect import currentframe

class MomentOfInertia:
#####Before adding particle tag and updating moment within function. The commented out code 
#####is just a backup!
#     #Principle Moment Calculator
#     def Moment(self, coord, Mass, debug=None):
#         DB = Debug()
#         #Referencing local variables
        
#         cx,cy,cz = 0.0,0.0,0.0
#         x,y,z = 0.0,0.0,0.0
#         cXYZ = []
#         TotalMass = sum(Mass)
#         if(debug==True):
#             print("DEBUG: Moment: len(coord)= ", len(coord))
#             print("DEBUG: Moment: TotalMass = ", TotalMass)
#         for i in range(len(Mass)):
#                 x += Mass[i]*coord[i][0]
#                 y += Mass[i]*coord[i][1]
#                 z += Mass[i]*coord[i][2]
#                 if(debug==True):
#                     print("DEBUG: Moment: x = Mass[%d]"%i +"*coord[%d][0]"%i +" =%f "%x)
#                     print("DEBUG: Moment: y = Mass[%d]"%i +"*coord[%d][1]"%i +" =%f "%y)
#                     print("DEBUG: Moment: z = Mass[%d]"%i +"*coord[%d][2]"%i +" =%f "%z)

#         cx = x/TotalMass
#         cy = y/TotalMass
#         cz = z/TotalMass
#         if(debug==True):
#             print("DEBUG: Moment: cx = x/TotalMass = ", cx)
#             print("DEBUG: Moment: cy = y/TotalMass = ", cy)
#             print("DEBUG: Moment: cz = y/TotalMass = ", cz)
#         com = [cx,cy,cz]
#         Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        
#         for i in range(len(coord)):
#             cXYZ.append([coord[i][0]-com[0], coord[i][1]-com[1], coord[i][2]-com[2]]) 
#         if(debug==True):
#             print("DEBUG: Moment: cXYZ = ", cXYZ)
#             print("DEBUG: Moment: len(cXYZ) = ", len(cXYZ))
#         Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        
#         #Constructs Inertia Tensor
#         #Useful Property: Ixy=Iyx, Ixz=Izx, Iyz=Izy
#         for i in range(len(coord)):
#             Ixx += Mass[i]*(cXYZ[i][1]**2 + cXYZ[i][2]**2)
#             Iyy += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][2]**2)
#             Izz += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][1]**2)
#             Ixy += -Mass[i]*cXYZ[i][0]*cXYZ[i][1]
#             Ixz += -Mass[i]*cXYZ[i][0]*cXYZ[i][2]
#             Iyz += -Mass[i]*cXYZ[i][1]*cXYZ[i][2]
            
#         #Computes eignevalues, finds principle moments
#         Imatrix = np.matrix([[Ixx,Ixy,Ixz],[Ixy,Iyy,Iyz],[Ixz,Iyz,Izz]])
#         Idiag = np.linalg.eig(Imatrix)
#         print("DEBUG: Moment: Idiag = ",Idiag)
#         comX = 0-com[0]
#         comY = 0-com[1] 
#         for i in range(len(coord)):
#             coord[i] = [coord[i][0]+comX,coord[i][1]+comY,coord[i][2]]
#         #return coord

  
    #Principle Moment Calculator
    def Moment(self, coord, Mass, rigid_tag=None, debug=None):
        DB = Debug()
        #Referencing local variables
        cx,cy,cz = 0.0,0.0,0.0
        x,y,z = 0.0,0.0,0.0
        cXYZ = []
        TotalMass = sum(Mass)
        if(debug==True):
            print("DEBUG: Moment: len(coord)= ", len(coord))
            print("DEBUG: Moment: TotalMass = ", TotalMass)
        for i in range(len(Mass)):
                x += Mass[i]*coord[i][0]
                y += Mass[i]*coord[i][1]
                z += Mass[i]*coord[i][2]
                if(debug==True):
                    print("DEBUG: Moment: x = Mass[%d]"%i +"*coord[%d][0]"%i +" =%f "%x)
                    print("DEBUG: Moment: y = Mass[%d]"%i +"*coord[%d][1]"%i +" =%f "%y)
                    print("DEBUG: Moment: z = Mass[%d]"%i +"*coord[%d][2]"%i +" =%f "%z)

        cx = x/TotalMass
        cy = y/TotalMass
        cz = z/TotalMass
        if(debug==True):
            print("DEBUG: Moment: cx = x/TotalMass = ", cx)
            print("DEBUG: Moment: cy = y/TotalMass = ", cy)
            print("DEBUG: Moment: cz = y/TotalMass = ", cz)
        com = [cx,cy,cz]
        Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        
        for i in range(len(coord)):
            cXYZ.append([coord[i][0]-com[0], coord[i][1]-com[1], coord[i][2]-com[2]]) 
        if(debug==True):
            print("DEBUG: Moment: cXYZ = ", cXYZ)
            print("DEBUG: Moment: len(cXYZ) = ", len(cXYZ))
        Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        
        #Constructs Inertia Tensor
        #Useful Property: Ixy=Iyx, Ixz=Izx, Iyz=Izy
        for i in range(len(coord)):
            Ixx += Mass[i]*(cXYZ[i][1]**2 + cXYZ[i][2]**2)
            Iyy += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][2]**2)
            Izz += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][1]**2)
            Ixy += -Mass[i]*cXYZ[i][0]*cXYZ[i][1]
            Ixz += -Mass[i]*cXYZ[i][0]*cXYZ[i][2]
            Iyz += -Mass[i]*cXYZ[i][1]*cXYZ[i][2]
        if(debug==True):
            print("DEBUG: Moment: Ixx = ", Ixx)
            print("DEBUG: Moment: Iyy = ", Iyy)
            print("DEBUG: Moment: Izz = ", Izz)
            print("DEBUG: Moment: Ixy = ", Ixy)
            print("DEBUG: Moment: Ixz = ", Ixz)
            print("DEBUG: Moment: Iyz = ", Iyz)

        #Computes eignevalues, finds principle moments
        Imatrix = np.matrix([[Ixx,Ixy,Ixz],[Ixy,Iyy,Iyz],[Ixz,Iyz,Izz]])
        Idiag = np.linalg.eig(Imatrix)
        
        #Sets the moment of inertia for the rigid body
        if(rigid_tag!=None):
            system.particles.get(rigid_tag).moment_intertia = Idiag[0]
        #system.particles[rigid_tag].moment_intertia = [Ixx, Iyy, Izz]
        if(debug==True):
            print("DEBUG: Moment: Idiag = ",Idiag)
            print("DEBUG: Moment: Idiag[0] = ",Idiag[0])
            print("DEBUG: MOMENT: print(system.particles[%d])"%rigid_tag)
            if(rigid_tag!=None or rigid_tag!=False):
                print(system.particles[rigid_tag])
        comX = 0-com[0]
        comY = 0-com[1] 
        for i in range(len(coord)):
            coord[i] = [coord[i][0]+comX,coord[i][1]+comY,coord[i][2]]
        return Idiag[0], Imatrix
    

        

In [32]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 
from inspect import currentframe

#Number of rigid bodies
#nrigid = 2
#Box Size
boxsize = 1
#Number of constituent particles
nparticles = 10
#Total number of patches in sim
tot_npatch = 4
#Rigid Diameter
diam = 1.0
#List of Patch Diameters
pdiam = [1.0]
#LJ Cutoff
rcut = 2.5
#Random Seed
SEED = np.random.randint(0,100000000)
##----------SETUP----------##
context.initialize("")

def Angle(angle):
    angle = angle*np.pi/180
    return angle

###Placeholers for sig and eps values

# sig = [1.0]*(nparticles+tot_npatch)
# eps = [0.1]*(nparticles+tot_npatch)
sig = [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.5,1.5,1.5,1.5]
eps = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.25,0.25,0.25,0.25]
print("sig = ", sig)
print("eps = ", eps)

DB = Debug()
DB.DebugInfo()
SB = SimBox()
MoI = MomentOfInertia()
Poly = Polymer()

#snapshot = SB.MakeSnapshot(boxsize, nparticles)
snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(Lx=11, Ly=4, Lz=4),
        bond_types=['tether'],particle_types=SB.GenParticleTypes(nparticles, debug=True))

system = hoomd.init.read_snapshot(snapshot)


##Adds patch Patch Types from [1,tot_npatch]
Poly.PatchTypes(tot_npatch, debug=True)


#types = system.particles.types
#DB.DebugMessage("DEBUG: types: ",types, debug=True)


Poly.SetPolyProperties(nparticles, boxsize, diam=1.0, debug=True)
# system.particles.get(0).moment_intertia = [0.611524 , 1.1476125, 1.7591365]
# system.particles.get(0).diameter = 1.0

rigid=hoomd.md.constrain.rigid()


##----------Rigid Body 1----------##
#Patch group properties
coord_patch_group_1 = [[np.cos(Angle(60)),0,np.sin(Angle(60))],[np.cos(Angle(60)),0,-np.sin(Angle(60))]]
mass_patch_group_1 = [1.0,1.0]
types_patch_group_1 = ['1','2']
diameters_patch_group_1 = [1.0,1.0]
rigid.set_param('C0',positions=coord_patch_group_1, types=['1','2'], diameters = [1.0,1.0])

#Calculates and sets moment of inertia. 
eigen, tensor = MoI.Moment(coord_patch_group_1, mass_patch_group_1, rigid_tag=False, debug=False)
DB.DebugMessage("DEBUG: eigen = ",eigen, debug=True)
DB.DebugMessage("DEBUG: tensor = ",tensor, debug=True)


rigid.set_param('C0',positions=coord_patch_group_1, types=['1','2'], diameters = [1.0,1.0])

##----------Rigid Body 2----------##
#Patch group properties
coord_patch_group_2 = [[np.cos(Angle(60)),0,np.sin(Angle(60))],[np.cos(Angle(60)),0,-np.sin(Angle(60))]]
mass_patch_group_2 = [1.0,1.0]
types_patch_group_2 = ['3','4']
diameters_patch_group_2 = [1.0,1.0]

#Calculates and sets moment of inertia
MoI.Moment(coord_patch_group_2, mass_patch_group_2, rigid_tag=False,debug=False)
rigid.set_param('C9',positions=coord_patch_group_2, types=['3','4'], diameters = [1.0,1.0])

rigid.create_bodies()

# system.particles[0].moment_intertia = eigen
# DB.DebugMessage("DEBUG: system.particles[0]",system.particles[0], debug=True)


Poly.MakeBonds(nparticles, debug=False)

system.replicate(nx=5,ny=5,nz=5)

types = system.particles.types

harm = md.bond.harmonic()
harm.bond_coeff.set('tether',k=20,r0=0.83)

#Groups 
center = hoomd.group.rigid_center()
nonrigid = hoomd.group.nonrigid()
part = hoomd.group.all()

system.particles[0].moment_intertia = eigen
system.particles.get(0).moment_intertia = eigen
DB.DebugMessage("DEBUG: system.particles[0]",system.particles[0], debug=True)
#DB.PrintSysParticles(debug=False)




nl = md.nlist.cell(check_period = 1)
lj = md.pair.lj(r_cut=rcut, nlist=nl)

#Calculates LJ forces (this might be causing problems)
for i in range(0,len(sig)):
    for j in range(i,len(sig)):
        lj.pair_coeff.set(types[i],types[j],epsilon=0.5*(eps[i]+eps[j]),sigma=0.5*(sig[i]+sig[j]))
        
        
        #Integrators. 
hoomd.md.integrate.mode_standard(dt=0.002)
#hoomd.md.integrate.langevin(group=part, kT=0.1, seed = 5)
hoomd.md.integrate.langevin(group=nonrigid, kT=0.1, seed = SEED)
hoomd.md.integrate.langevin(group=center, kT=0.1, seed = SEED)

#Allows me to make unique file names
t = time.strftime(time.strftime("%d %b %H:%M:%S", time.gmtime()))

### DUMP AND RUN ###
hoomd.analyze.log(filename='random.log',quantities=['temperature','num_particles','ndof',
                                                    'translational_ndof','rotational_ndof',
                                                    'potential_energy'],period=1000,header_prefix='#'
                                                      ,overwrite=True)
#hoomd.deprecated.dump.xml(group=part,filename ='random.xml',vis=True,image=True)
hoomd.dump.gsd(filename='random %s.gsd'%t,period=1000, group = hoomd.group.all() ,overwrite = True, dynamic=['attribute'])
hoomd.run(1e5)

sig =  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.5, 1.5, 1.5, 1.5]
eps =  [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.25, 0.25, 0.25, 0.25]
Debug Message Format Is As Follows:
 DEBUG: Function: Parameter = Value
DEBUG: GenParticleTypes: Particle Types =  ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
notice(2): Group "all" created containing 10 particles
DEBUG: PatchTypes:  []
DEBUG: SetPolyProperties: Particle 0 is at  [10, 0, 0]
DEBUG: SetPolyProperties: Particle 1 is at  [11, 0, 0]
DEBUG: SetPolyProperties: Particle 2 is at  [12, 0, 0]
DEBUG: SetPolyProperties: Particle 3 is at  [13, 0, 0]
DEBUG: SetPolyProperties: Particle 4 is at  [14, 0, 0]
DEBUG: SetPolyProperties: Particle 5 is at  [15, 0, 0]
DEBUG: SetPolyProperties: Particle 6 is at  [16, 0, 0]
DEBUG: SetPolyProperties: Particle 7 is at  [17, 0, 0]
DEBUG: SetPolyProperties: Particle 8 is at  [18, 0, 0]
DEBUG: SetPolyProperties: Particle 9 is at  [19, 0, 0]
DEBUG: eigen =  [1.5 1.5 0. ]  at li

*Warning*: Integrator #0: Anisotropic integration requested, but no rotational degrees of freedom found for its group
*Warning*: Integrator #1: Anisotropic integration requested, but no rotational degrees of freedom found for its group


Time 00:00:10 | Step 15815 / 100000 | TPS 1581.47 | ETA 00:00:53
Time 00:00:20 | Step 29107 / 100000 | TPS 1329.12 | ETA 00:00:53
Time 00:00:30 | Step 42049 / 100000 | TPS 1294.17 | ETA 00:00:44
Time 00:00:40 | Step 54762 / 100000 | TPS 1271.3 | ETA 00:00:35
Time 00:00:50 | Step 67101 / 100000 | TPS 1233.73 | ETA 00:00:26
Time 00:01:00 | Step 78402 / 100000 | TPS 1130.04 | ETA 00:00:19
Time 00:01:10 | Step 89442 / 100000 | TPS 1104 | ETA 00:00:09
Time 00:01:18 | Step 100000 / 100000 | TPS 1213.3 | ETA 00:00:00
Average TPS: 1270.54
---------
-- Neighborlist stats:
997 normal updates / 1000 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 48 / n_neigh_avg: 16.1731
shortest rebuild period: 62
-- Cell list stats:
Dimension: 18, 6, 6
n_min    : 0 / n_max: 29 / n_avg: 2.70062
** run complete **


In [74]:
print(system.particles[0])
system.particles[0].moment_inertia=[0.611524, 1.1476125, 1.7591365]
print(system.particles[0])

tag         : 0
position    : (-12.998936415526476, -7.999294073341945, -7.999610284932579)
image       : (0, 0, 0)
velocity    : (0.09700008139675893, 0.14257481128721933, 0.022772407711068615)
acceleration: (-12.856825577140887, 14.266452577780601, 5.49126644643195)
charge      : 0.0
mass        : 1.0
diameter    : 1.0
type        : C0
typeid      : 0
body        : 0
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.0, 0.0, 0.0)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (1.0697678232242613, -0.0008758984411417897, 0.0013922121470782362)
net_energy  : -0.3013848684010576
net_torque  : (4.290946244154516e-05, -0.0010693707324341828, 0.0)
net_virial  : (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

tag         : 0
position    : (-12.998936415526476, -7.999294073341945, -7.999610284932579)
image       : (0, 0, 0)
velocity    : (0.09700008139675893, 0.14257481128721933, 0.022772407711068615)
acceleration: (-12.856825577140887, 14.266452577780601, 5.49126644643195)
charge      : 0.0
mass    

In [56]:
DB.PrintSysParticles(debug=True)

tag         : 0
position    : (-9.576418092212533, 9.582002704294307, -9.337701442160396)
image       : (0, -1, 0)
velocity    : (0.1677113004619233, -0.34037389549233976, -0.20090152484269883)
acceleration: (-2.4489325489824307, -2.344907128140413, 0.28520571634684366)
charge      : 0.0
mass        : 1.0
diameter    : 1.0
type        : C0
typeid      : 0
body        : 0
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.0, 0.0, 0.0)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (1.3203357372936702, 0.3058853214853705, -0.9221275123141383)
net_energy  : -1.5510775506357115
net_torque  : (-0.7853220985002136, 0.7186346753742788, 0.0)
net_virial  : (-1.4422186756802318, -0.19120397017124996, -0.6435512527392167, 0.3017988154514275, -0.4770455441668393, -2.302655718545767)

tag         : 1
position    : (-8.802165561815794, -9.834853257410064, -9.380261933163181)
image       : (0, 0, 0)
velocity    : (0.17108544092004035, 0.06192126197628836, -0.4942069233888892)
acceleration: (17


tag         : 1368
position    : (23.420915027174168, 6.229854748288062, 2.7188071281624744)
image       : (0, 0, 0)
velocity    : (0.0, 0.0, 0.0)
acceleration: (0.0, 0.0, 0.0)
charge      : 0.0
mass        : 1.0
diameter    : 1.0
type        : 1
typeid      : 10
body        : 1358
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.0, 0.0, 0.0)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (0.0, 0.0, 0.0)
net_energy  : 0.0
net_torque  : (0.0, 0.0, 0.0)
net_virial  : (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

tag         : 1369
position    : (23.420915027174168, 6.229854748288062, 0.9867563205935974)
image       : (0, 0, 0)
velocity    : (0.0, 0.0, 0.0)
acceleration: (0.0, 0.0, 0.0)
charge      : 0.0
mass        : 1.0
diameter    : 1.0
type        : 2
typeid      : 11
body        : 1358
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.0, 0.0, 0.0)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (0.0, 0.0, 0.0)
net_energy  : 0.0
net_torque  : (0.0, 0.0, 0.0)
net_virial  : (0.0, 0.0

In [ ]:
#     def GeneralDebug(self, string, variable, debug=None):
#         if(debug==True):
#             print("%s"%string, variable)

In [8]:
DB.GeneralDebug("DEBUG: DEBUG", XYZ, debug=True)

DEBUG: DEBUG [[(0.7071067811865476, -2.1213203435596424, 0), (-0.7071067811865476, -2.1213203435596424, 0)]]


In [124]:
print(XYZ)

[(0.7071067811865476, -2.1213203435596424, 0), (-0.7071067811865476, -2.1213203435596424, 0)]


In [12]:
print(coord[0][0])

[0.7071067811865476, -2.1213203435596424, 0]


In [120]:
coord[0][0]

0.7071067811865476

In [18]:
for i in range(nparticles+tot_npatch+2):
    print(system.particles[i].tag)

0
1
2
3
4
5
6


IndexError: 

In [16]:
print(types)

Particle types: ['C1', 'C2', 'C3', 'C4', 'C5', '1']


In [14]:
for i in range(0,len(sig)):
    for j in range(i,len(sig)):
        print([types[i],types[j]])
        
        
        
        
        
        

['C1', 'C1']
['C1', 'C2']
['C1', 'C3']
['C1', 'C4']
['C1', 'C5']
['C1', '1']
['C2', 'C2']
['C2', 'C3']
['C2', 'C4']
['C2', 'C5']
['C2', '1']
['C3', 'C3']
['C3', 'C4']
['C3', 'C5']
['C3', '1']
['C4', 'C4']
['C4', 'C5']
['C4', '1']
['C5', 'C5']
['C5', '1']
['1', '1']


In [24]:
for i in range(7):
    print(system.particles[i])

tag         : 0
position    : (0.7797972164629362, 1.1832019127317999, -0.8005314266409833)
image       : (0, 0, 0)
velocity    : (0.013986810484661497, 0.03473065968063528, -0.024355229000734127)
acceleration: (0.30145861751631603, 0.961126191606986, -1.4695685780248129)
charge      : 0.0
mass        : 1.0
diameter    : 1.0
type        : C1
typeid      : 0
body        : 0
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.0, 0.0, 0.0)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (0.13429209747312917, -0.1523564962800972, 0.004061617380839462)
net_energy  : 0.0005157920767278524
net_torque  : (0.0, 0.0, 0.0)
net_virial  : (-0.04484121146944117, 0.05087305952463784, -0.0013562067114084844, -0.057716285991984986, 0.001538637839974866, -4.101799625380054e-05)

tag         : 1
position    : (1.4476133026390205, 0.42555406178782085, -0.7803335633723744)
image       : (0, 0, 0)
velocity    : (-0.034307861215994814, -0.06587781654288995, -0.0036334899960951887)
acceleration: (-1.1231

IndexError: 

In [38]:
print(system.particles[1])

tag         : 1
position    : (-2.0, 0.0, 0.0)
image       : (1, 0, 0)
velocity    : (0.0, 0.0, 0.0)
acceleration: (0.0, 0.0, 0.0)
charge      : 0.0
mass        : 1.0
diameter    : 0.9
type        : C1
typeid      : 0
body        : 4294967295
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.0, 0.0, 0.0)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (0.0, 0.0, 0.0)
net_energy  : 0.0
net_torque  : (0.0, 0.0, 0.0)
net_virial  : (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)



In [39]:
np.cos(angle)


0.8660254037844387

In [82]:
class MomentOfInertia:

        #Coord [[x1,y1,z1],...,[xn,yn,zn]]
        #GetCoord[k] = [xk,yk,zk] where k ∈ n
        #GetCoord[k][0] = xk
    #Should work with arbitrary number of coordinates
    #coord = [[x1,y1,z1],...,[xn,yn,zn]]
    def Moment(self, coord, Mass, debug=None):
        DB = Debug()
        #Referencing local variables
        
        cx,cy,cz = 0.0,0.0,0.0
        x,y,z = 0.0,0.0,0.0
        cXYZ = []
        TotalMass = sum(Mass)
        if(debug==True):
            print("DEBUG: len(coord)= ", len(coord))
            print("DEBUG: TotalMass = ", TotalMass)
        for i in range(len(Mass)):
                x += Mass[i]*coord[i][0]
                y += Mass[i]*coord[i][1]
                z += Mass[i]*coord[i][2]
                if(debug==True):
                    print("DEBUG: x = Mass[%d]"%i +"*coord[%d][0]"%i +" =%f "%x)
                    print("DEBUG: y = Mass[%d]"%i +"*coord[%d][1]"%i +" =%f "%y)
                    print("DEBUG: z = Mass[%d]"%i +"*coord[%d][2]"%i +" =%f "%z)

        cx = x/TotalMass
        cy = y/TotalMass
        cz = z/TotalMass
        if(debug==True):
            print("DEBUG: cx = ", cx)
            print("DEBUG: cy = ", cy)
            print("DEBUG: cz = ", cz)
        com = [cx,cy,cz]
        Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        
        for i in range(len(coord)):
            cXYZ.append([coord[i][0]-com[0], coord[i][1]-com[1], coord[i][2]-com[2]]) 
        if(debug==True):
            print("DEBUG: cXYZ = ", cXYZ)
            print("DEBUG: len(cXYZ) = ", len(cXYZ))
        Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        for i in range(len(coord)):
            Ixx += Mass[i]*(cXYZ[i][1]**2 + cXYZ[i][2]**2)
            Iyy += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][2]**2)
            Izz += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][1]**2)
            Ixy += -Mass[i]*cXYZ[i][0]*cXYZ[i][1]
            Ixz += -Mass[i]*cXYZ[i][0]*cXYZ[i][2]
            Iyz += -Mass[i]*cXYZ[i][1]*cXYZ[i][2]
        Imatrix = np.matrix([[Ixx,Ixy,Ixz],[Ixy,Iyy,Iyz],[Ixz,Iyz,Izz]])
        Idiag = np.linalg.eig(Imatrix)
        print("DEBUG: Idiag = ",Idiag)
        comX = 0-com[0]
        comY = 0-com[1] 
        for i in range(len(coord)):
            coord[i] = [coord[i][0]+comX,coord[i][1]+comY,coord[i][2]]
        return Idiag[0]

In [83]:
MoI = MomentOfInertia()
#coord = [[np.cos(Angle(45)),np.sin(Angle(45)),0],[-np.cos(Angle(30)),np.sin(Angle(30)),0]]
coord = [[0,0,0],[0.7575,0.5865,0],[1.515,0,0]]
Mass = [1.0,16.0,1.0]

minert = MoI.Moment(coord, Mass,debug=True)

DEBUG: len(coord)=  3
DEBUG: TotalMass =  18.0
DEBUG: x = Mass[0]*coord[0][0] =0.000000 
DEBUG: y = Mass[0]*coord[0][1] =0.000000 
DEBUG: z = Mass[0]*coord[0][2] =0.000000 
DEBUG: x = Mass[1]*coord[1][0] =12.120000 
DEBUG: y = Mass[1]*coord[1][1] =9.384000 
DEBUG: z = Mass[1]*coord[1][2] =0.000000 
DEBUG: x = Mass[2]*coord[2][0] =13.635000 
DEBUG: y = Mass[2]*coord[2][1] =9.384000 
DEBUG: z = Mass[2]*coord[2][2] =0.000000 
DEBUG: cx =  0.7575
DEBUG: cy =  0.5213333333333333
DEBUG: cz =  0.0
DEBUG: cXYZ =  [[-0.7575, -0.5213333333333333, 0.0], [0.0, 0.0651666666666667, 0.0], [0.7575, -0.5213333333333333, 0.0]]
DEBUG: len(cXYZ) =  3
DEBUG: Idiag =  (array([0.611524 , 1.1476125, 1.7591365]), matrix([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]))


In [81]:
print(minert)

[0.611524  1.1476125 1.7591365]


In [84]:
system.particles[0].moment_inertia = minert

In [85]:
print(system.particles[0])

tag         : 0
position    : (-12.998936415526476, -7.999294073341945, -7.999610284932579)
image       : (0, 0, 0)
velocity    : (0.09700008139675893, 0.14257481128721933, 0.022772407711068615)
acceleration: (-12.856825577140887, 14.266452577780601, 5.49126644643195)
charge      : 0.0
mass        : 1.0
diameter    : 1.0
type        : C0
typeid      : 0
body        : 0
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.6115240000000001, 1.1476125, 1.7591364999999999)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (1.0697678232242613, -0.0008758984411417897, 0.0013922121470782362)
net_energy  : -0.3013848684010576
net_torque  : (4.290946244154516e-05, -0.0010693707324341828, 0.0)
net_virial  : (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)



In [89]:
m = int(len(coord))

In [90]:
print(type(m))

<class 'int'>


In [93]:
coord[0][1]

1.6832019127317999

In [36]:
test = [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.5,1.5,1.5,1.5]

In [39]:
print(test*2)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.5, 1.5, 1.5, 1.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.5, 1.5, 1.5, 1.5]


In [40]:
print(types)

Particle types: ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', '1', '2', '3', '4']


In [8]:


def get_linenumber():
    cf = currentframe()
    return cf.f_back.f_lineno

print("This is line 7, python says line ", get_linenumber())

This is line 7, python says line  7


In [14]:
def testvar123(param=None):
    if(param==None or param==False):
        print("pass")
    else:
        x = param**2
        return x

In [15]:
testvar123()

pass


In [47]:
#print("DEBUG: system.particles[0] = {0} at line number {1}".format([system.particles.get(0).moment_inertia],DB.linenumber()))
print("DEBUG: system.particles[0] = {0} \nat line number {1}".format(system.particles[0],DB.linenumber()))








DEBUG: system.particles[0] = tag         : 0
position    : (-12.999544387734145, -8.000738692374556, -7.999147266087841)
image       : (0, 0, 0)
velocity    : (-0.03723406837914466, -0.031155043789176898, 0.061496850099542325)
acceleration: (14.617523019935513, 11.629332129522497, -8.099392183002669)
charge      : 0.0
mass        : 1.0
diameter    : 1.0
type        : C0
typeid      : 0
body        : 0
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.0, 0.0, 0.0)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (1.1425940248943163, -7.375647253443255e-05, -0.03263836170861202)
net_energy  : -0.29991227441514867
net_torque  : (5.162956203399231e-05, 0.03002186742677293, 0.0)
net_virial  : (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
 
at line number 2


In [39]:
system.particles.get(0).moment_inertia

(0.0, 0.0, 0.0)